In [69]:
from ib_insync import IB, util, Future, Stock, Forex, ContFuture, Index, FutureOption

ImportError: cannot import name 'FutureOption' from 'ib_insync' (/Users/weston/.pyenv/versions/3.9.16/envs/finx-all/lib/python3.9/site-packages/ib_insync/__init__.py)

In [4]:
from ib_insync import IB, util

# required for jupyter notebook
util.startLoop()
ib = IB()

# mac
host = "127.0.0.1"

# if using WSL on Windows,
# $ ip route
# default via 172.21.32.1 dev eth0
# host = "172.21.32.1"  # got this from WSL ip route -> default value

# must set timeout to 30 seconds and then white list this IP in TWS
# expect to confirm network connection in TWS
ib.connect(host, 7496, clientId=3, timeout=30)

<IB connected to 127.0.0.1:7496 clientId=3>

In [16]:
from ib_insync import Index, Future, FuturesOption

# spx = Index("SPX", "CBOE")
# es = Future("ES", "202306", "CME")
# ib.qualifyContracts(es)

# sym = 'ES'
# contract = Future(sym, "202306", exchange='CME')

# For more context on fetching ES options see this discussion, https://groups.io/g/insync/topic/96448496#8746
es = Index("ES", "CME", "USD")
ib.qualifyContracts(es)
chains = ib.reqSecDefOptParams("ES", "SMART", "IND", es.conId)
df = util.df(chains)
# df = df.explode('expirations')
# df = df.reset_index(drop=True)

In [34]:
es_future = Future("ES", "202306", "CME")
ib.qualifyContracts(es_future)
# es_future
ib.reqMarketDataType(1)
[ticker] = ib.reqTickers(es_future)

In [44]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np


def future_close(symbol="ES"):
    es_future = Future(symbol, "202306", "CME")
    ib.qualifyContracts(es_future)
    ib.reqMarketDataType(1)
    [ticker] = ib.reqTickers(es_future)
    return ticker.close


def get_expirations_within_x_days(option_chain_df, days=45):
    # flatten values of lists
    expiration_strs = pd.Series(np.concatenate(option_chain_df.expirations))
    expirations = pd.to_datetime(expiration_strs, format="%Y%m%d")
    max_date = datetime.today() + timedelta(days=days)
    return expirations[expirations < max_date]


def strike_min_max(price, bps=100.0):
    if price is None or np.isnan(price):
        raise ValueError("price must be greater than 0.0")
    diff = price * (bps / 10000.0)
    return price - diff, price + diff


def strikes_within_x_bps(option_chain_df, price, bps=100.0):
    strike_strs = pd.Series(np.concatenate(option_chain_df.strikes))
    strikes = pd.to_numeric(strike_strs, errors="coerce")
    lower, upper = strike_min_max(price, bps)
    return sorted(strikes[(lower <= strikes) & (strikes <= upper)].unique())


def get_close_price(contract):
    [ticker] = ib.reqTickers(contract)
    return ticker.close

In [101]:
# close_price = get_close_price(es)
close_price = future_close("ES")

strikes_width_bps = 100
strikes = strikes_within_x_bps(df, close_price, strikes_width_bps)

rights = ["C", "P"]

df = df.sort_values(["expirations"])
df["FutureOptions"] = None

In [133]:
from ib_insync import FuturesOption

lower, upper = strike_min_max(close_price, 40)
print(f"close={close_price} +- {strikes_width_bps}bps, [{lower}, {upper}]")


today = datetime.today()

for i, row in df.iterrows():
    strikes = [float(x) for x in row.strikes]
    strikes_focused = [s for s in strikes if lower <= s and s <= upper]

    for expiration in row.expirations:
        date = datetime.strptime(expiration, "%Y%m%d")
        if date < today or date > today + timedelta(days=10):
            continue
        dte = (date - today).days

        print(expiration, f"dte={dte}")
        print(f"strikes_focused, len={len(strikes_focused)}")
        fops = []
        for strike in strikes_focused:
            for right in rights:
                fop = FuturesOption(
                    "ES", expiration, strike, right, "CME", row.multiplier
                )
                fops.append(fop)

        ib.qualifyContracts(*fops)
        df.loc[i, "FutureOptions"] = fops

close=4171.5 +- 100bps, [4154.814, 4188.186]
20230518 dte=0
strikes_focused, len=7
20230519 dte=1
strikes_focused, len=7
20230522 dte=4
strikes_focused, len=7
20230523 dte=5
strikes_focused, len=7
20230524 dte=6
strikes_focused, len=7
20230525 dte=7
strikes_focused, len=4
20230526 dte=8
strikes_focused, len=4


In [139]:
for i, row in df[df.FutureOptions.notnull()].iterrows():
    v = row.FutureOptions
    ib.reqMarketDataType(4)
    tickers = ib.reqTickers(*v)

    ivs = []
    for t in tickers:
        iv = t.modelGreeks.impliedVol
        ivs.append(iv)
        # print(f"{iv:0.3f} {t.contract.right} {t.contract.strike} {t.contract.lastTradeDateOrContractMonth}")

    # df['ivs'] = ivs
    df.loc[i, "iv_mean"] = np.mean(ivs) if len(ivs) > 0 else None

In [140]:
df

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes,FutureOpions,FutureOptions,iv_mean
10,CME,495512569,E3B,50,[20230516],"[900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0...",None,None,0.128412
13,CME,495512569,E3C,50,[20230517],"[100.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0,...",None,None,0.128412
12,CME,495512569,E3D,50,"[20230518, 20230615]","[100.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0,...",None,"[FuturesOption(conId=630343091, symbol='ES', l...",0.155449
48,CME,495512569,EW3,50,[20230519],"[100.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0, ...",None,"[FuturesOption(conId=630413164, symbol='ES', l...",0.146163
20,CME,495512569,E4A,50,[20230522],"[900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0...",None,"[FuturesOption(conId=631187727, symbol='ES', l...",0.114185
19,CME,495512569,E4B,50,[20230523],"[900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0...",None,"[FuturesOption(conId=631171013, symbol='ES', l...",0.118509
18,CME,495512569,E4C,50,[20230524],"[900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0...",None,"[FuturesOption(conId=631183201, symbol='ES', l...",0.122225
21,CME,495512569,E4D,50,[20230525],"[100.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0,...",None,"[FuturesOption(conId=626454267, symbol='ES', l...",0.125374
46,CME,495512569,EW4,50,[20230526],"[100.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0, ...",None,"[FuturesOption(conId=617730710, symbol='ES', l...",0.128320
34,CME,495512569,E5B,50,[20230530],"[900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0...",None,None,0.128412


In [138]:
ddk

In [159]:
from ib_insync import FuturesOption

# raise NotImplementedError("This is not working yet - create futures options for each row - ")

contracts = [
    FuturesOption("ES", expiration.strftime("%Y%m%d"), strike, right, "CME", "50")
    for right in rights
    for expiration in expiration_dates
    for strike in strikes
]

ib.qualifyContracts(*contracts)

[]

In [152]:
len(contracts)

952

In [102]:
# ib.reqMarketDataType(1)
df.sort_values("expirations", ascending=True).head(1)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
10,CME,495512569,E3B,50,[20230516],"[900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0..."


In [132]:
from ib_insync import FuturesOption


expiration = "20230516"  # expiration_dates.sort_values(ascending=True).values[0]
call_strikes = strikes_within_x_bps(df, close_price, 100)
op_contracts = [
    FuturesOption(es.symbol, expiration, strike, "C", "CME", "50")
    for strike in call_strikes
]

ib.qualifyContracts(*op_contracts)

len(op_contracts)

17

In [134]:
tickers = ib.reqTickers(*op_contracts)

In [148]:
pd.set_option("display.max_rows", 100)
# util.df(tickers).head(1).T

raise NotImplementedError("start from here")

# tickers[0].modelGreeks.impliedVol

In [ ]:
strikes_within_x_points = 50

strikes = [
    strike
    for strike in chain.strikes
    if strike % 5 == 0
    and spxValue - strikes_within_x_points < strike < spxValue + strikes_within_x_points
]


import pandas as pd
from datetime import datetime

today = datetime.today()
dt = pd.to_datetime(today) + pd.DateOffset(days=20)
expirations = sorted(exp for exp in chain.expirations)  # about 30 days out
expirations = [
    exp for exp in expirations if pd.to_datetime(exp) < dt
]  # remove weekly options
rights = ["P", "C"]

from ib_insync import Option

contracts = [
    Option("SPXW", expiration, strike, right, "SMART", tradingClass="SPXW")
    for right in rights
    for expiration in expirations
    for strike in strikes
]
len(contracts)

In [ ]:
contracts = ib.qualifyContracts(*contracts)

In [ ]:
contracts[0]

In [ ]:
tickers = ib.reqTickers(*contracts)

In [ ]:
# tickers[0].modelGreeks # market is open and live bid/asks are available
tickers[-1]

In [ ]:
from datetime import datetime

rows = []

today = datetime.today().date()
for ticker in tickers:
    date_str = ticker.contract.lastTradeDateOrContractMonth
    expiration_date = datetime.strptime(date_str, "%Y%m%d").date()
    dte = (expiration_date - today).days
    rows.append(
        [
            ticker.contract.strike,
            ticker.contract.right,
            dte,
            expiration_date,
            ticker.close,
        ]
    )

In [ ]:
import pandas as pd

odf = pd.DataFrame(rows, columns=["strike", "right", "dte", "expiration_date", "close"])

In [ ]:
# from finx_opion_pricer import Option
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
print(currentdir)
parentdir = os.path.dirname(currentdir)
print(parentdir)
sys.path.insert(0, "/home/weston/work/lsc/finx-ib-option-pricer")
# sys.path

In [ ]:
import pandas as pd

odf = pd.DataFrame(rows, columns=["strike", "right", "dte", "expiration_date", "close"])

from finx_option_pricer.option import Option

odf["iv_close"] = None
odf["delta"] = None

spxValue = 4137

for i, row in odf.iterrows():
    option_type = row["right"].lower()
    o = Option(
        S=spxValue,
        K=row["strike"],
        r=0.0,
        T=row["dte"] / 252,
        sigma=None,
        option_type=option_type,
    )
    iv = o.iv(row["close"])
    odf.loc[i, "iv_close"] = iv
    o = Option(
        S=spxValue,
        K=row["strike"],
        r=0.0,
        T=row["dte"] / 252,
        sigma=iv,
        option_type=option_type,
    )
    delta = o.delta
    odf.loc[i, "delta"] = delta

odf = odf[odf.close > 1.0].copy()

In [ ]:
odf["expiration_date"] = pd.to_datetime(odf["expiration_date"])
odf["expiration_date_daynum"] = pd.to_datetime(odf["expiration_date"]).dt.weekday
odf["expiration_date_weekday"] = pd.to_datetime(odf["expiration_date"]).dt.day_name()

In [ ]:
fomc_date_sd = pd.to_datetime("2023-05-02")
from datetime import timedelta

fomc_date_ed = fomc_date_sd + timedelta(days=3)

dte_fomc_sd = (fomc_date_sd.date() - today).days + 1
dte_fomc_ed = (fomc_date_ed.date() - today).days

april_opex = pd.to_datetime("2023-04-21")
dte_opexp_sd = (april_opex.date() - today).days + 1
dte_opexp_ed = (april_opex.date() - today).days

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

colors = np.where(odf["right"] == "C", "b", "-")
colors[
    (odf["right"] == "C")
    & (
        odf.expiration_date_daynum.isin(
            [
                0,
            ]
        )
    )
] = "green"

colors[odf["right"] == "P"] = "yellow"
colors[
    (odf["right"] == "P")
    & (
        odf.expiration_date_daynum.isin(
            [
                0,
            ]
        )
    )
] = "red"

odf.iv_close = odf.iv_close.astype(float)
xdf = odf.copy()
xdf["expiration_date_weekday"] = pd.to_datetime(xdf["expiration_date"]).dt.day_name()

fig = px.scatter(
    xdf,
    x="dte",
    y="iv_close",
    color="expiration_date_weekday",
    size="iv_close",
    hover_data=["strike", "right", "expiration_date"],
    width=1500,
    height=800,
)

fig.add_vrect(
    x0=dte_fomc_sd,
    x1=dte_fomc_ed,
    fillcolor="green",
    opacity=0.25,
    line_width=2,
    annotation_text="FOMC May2-3",
    annotation_position="top left",
)

fig.add_vrect(
    x0=dte_opexp_sd,
    x1=dte_opexp_ed,
    fillcolor="red",
    opacity=0.25,
    line_width=2,
    annotation_text="Arpril Opex (21)",
    annotation_position="top left",
)

In [ ]:
import pandas as pd

odf = pd.DataFrame(rows, columns=["strike", "right", "dte", "expiration_date", "close"])

from finx_option_pricer.option import Option

odf["iv_close"] = None
odf["delta"] = None


for i, row in odf.iterrows():
    option_type = row["right"].lower()
    o = Option(
        S=spxValue,
        K=row["strike"],
        r=0.0,
        T=row["dte"] / 252,
        sigma=None,
        option_type=option_type,
    )
    iv = o.iv(row["close"])
    odf.loc[i, "iv_close"] = iv
    o = Option(
        S=spxValue,
        K=row["strike"],
        r=0.0,
        T=row["dte"] / 252,
        sigma=iv,
        option_type=option_type,
    )
    delta = o.delta
    odf.loc[i, "delta"] = delta

odf = odf[odf.close > 1.0].copy()

px.scatter(odf[(odf.dte == 2)], x="strike", y="iv_close", color="right")

In [ ]:
atm = odf[odf.strike.between(spxValue - 10, spxValue + 10)]
atm = (
    atm.groupby(["dte"])
    .median()
    .reset_index()
    .drop(["close", "strike", "expiration_date_daynum"], axis=1)
)
# atm = atm.rename(columns={'right': 'option_type'})
# atm.option_type = atm.option_type.str.lower()
atm["underlying"] = "es"

today = datetime.today().strftime("%Y-%m-%d")
fn = f"es_atm.{today}.csv"
atm.to_csv(fn, index=False)

In [ ]:
ivdf = pd.read_csv(fn)


result = {}

for ix, row in ivdf.iterrows():
    result[ix] = round(row["iv_close"], 3)

result